In [1]:
import pandas as pd
import math
pd.set_option('display.max_columns', 500)

In [2]:
def arm(c, f, default=4):
    if math.isnan(c) and math.isnan(f):
        return default
    if math.isnan(c) and not math.isnan(f):
        return f
    if not math.isnan(c) and math.isnan(f):
        return c
    return min(f, c)

In [3]:
def batter_stats(year):
    traj = pd.read_csv(f"Trajectory/{year}_Trajectory.csv").set_index("Unnamed: 0")
    contact = pd.read_csv(f"Contact/{year}_Contact.csv").set_index("Unnamed: 0")
    power = pd.read_csv(f"Power/{year}_Power.csv").set_index("Unnamed: 0")
    speed = pd.read_csv(f"Run/{year}_Run.csv").set_index("Unnamed: 0")
    c_arm = pd.read_csv(f"Arm/Catchers/{year}_Catcher_Arm.csv").set_index("Unnamed: 0")
    f_arm = pd.read_csv(f"Arm/Fielders/{year}_Fielder_Arm.csv").set_index("Unnamed: 0")
    field = pd.read_csv(f"Fielding/{year}_Fielding.csv").set_index("MLBID")
    error = pd.read_csv(f"Error/{year}_ER.csv").set_index("MLBID")
    ability = pd.read_csv(f"Abilities/{year}_Hitter_Ability.csv").set_index('player_id')
    positions = pd.read_csv(f"Fielding/{year}_Positions.csv").set_index('MLBID')

    merge = (traj.merge(contact, how='inner', left_index=True, right_index = True)
             .merge(power, how='inner', left_index=True, right_index = True)
             .merge(speed, how='inner', left_index=True, right_index = True)
             .merge(c_arm, how='left', left_index=True, right_index = True)
             .merge(f_arm, how='left', left_index=True, right_index = True)
             .merge(field, how='left', left_index=True, right_index = True)
             .merge(error, how='left', left_index=True, right_index = True)
             .merge(positions, how='left', left_index=True, right_index = True))
    merge['Arm Strength'] = merge.apply(lambda x: arm(x['Arm Strength_x'], x['Arm Strength_y']), axis=1)
    merge['Fielding'] = merge['Fielding'].fillna(7)
    merge['Error Resistance'] = merge['Error Resistance'].fillna(7)
    merge = merge.loc[:,~merge.columns.duplicated()].copy()
    merge = merge.drop_duplicates()
    columns = ['PLAYERNAME_x', 'Trajectory', 'Contact', 'Power', 'Run Speed', 'Arm Strength', 'Fielding', 'Error Resistance', 'C', '1B', '2B', '3B', 'SS', 'LF', 'CF', 'RF']
    return merge[columns].fillna('F').merge(ability, how='left', left_index=True, right_index = True).fillna(False)

batters = batter_stats(2022)
batters.head()

,PLAYERNAME_x,Trajectory,Contact,Power,Run Speed,Arm Strength,Fielding,Error Resistance,C,1B,2B,3B,SS,LF,CF,RF,AggressiveHitter,PatientHitter,FreeSwinger,ContactHitter,PowerHitter,PullHitter,PushHitter,SprayHitter,ToughOut,Slugger,SlapHitter,Refined,GoodCatcher,GreatCatcher,GdIFHtr,GrtIFHtr,PivotMan
405395,Albert Pujols,3,9,213,4,4.0,9.0,12.0,F,A,F,F,F,F,F,F,False,False,False,False,True,True,False,False,False,False,False,False,False,False,False,False,False
408234,Miguel Cabrera,2,7,110,5,4.0,7.0,7.0,F,F,F,F,F,F,F,F,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
425877,Yadier Molina,3,8,118,4,9.0,11.0,7.0,A,F,F,F,F,F,F,F,True,False,False,False,False,False,False,False,False,False,True,False,True,False,False,False,False
429664,Robinson Cano,2,9,80,5,9.0,8.0,12.0,F,F,A,F,F,F,F,F,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
435559,Kurt Suzuki,3,9,128,5,7.0,9.0,7.0,A,F,F,F,F,F,F,F,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [13]:
batters.to_csv("Batters_Data.csv")